In [1]:
import pandas as pd
import numpy as np
import time
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

In [2]:
url = "https://www.collegedata.com/cs/search/college/college_search_tmpl.jhtml"

In [36]:
profile = webdriver.FirefoxProfile()

profile.set_preference("dom.disable_open_during_load", False)
driver = webdriver.Firefox(firefox_profile=profile)

driver = webdriver.Firefox()
driver.get(url)

## SHRINK THE PAGE BEFORE HITTING RETURN ON THE NEXT LINES

In [37]:
button = driver.find_element_by_name("institutionType")

In [38]:
# This just makes sure we got the right button
html = button.get_attribute('outerHTML')
html

'<select name="institutionType" class="sel singleLineSel"><option value="">No Preference</option>\n<option value="2">Private</option>\n<option value="1">Public</option>\n<option value="3">Private for-profit</option></select>'

In [39]:
button = driver.find_element_by_name("institutionType")

In [40]:
button

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="0423d527-cdf7-3e44-ac6f-62b84884ceea", element="60120916-f434-a245-9f71-63fd65e41dfd")>

In [41]:
allOptions = button.find_elements_by_tag_name("option")
for option in allOptions:
    value = option.get_attribute("value")
    if value == '1':
        option.click()
        break

In [42]:
button = driver.find_element_by_name("states")
button

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="0423d527-cdf7-3e44-ac6f-62b84884ceea", element="87a9583d-0c55-7a41-8c6f-b76821585437")>

In [43]:
allOptions = button.find_elements_by_tag_name("option")
for option in allOptions:
    value = option.get_attribute("value")
    if value == 'WA':
        option.click()
        break

In [44]:

buttons = driver.find_elements_by_tag_name('input')

In [45]:
len(buttons)

106

In [46]:
# We're searching for a button that doesn't have a name set
# there are two find.gif buttons on the page and they each do 
# different things
for b in buttons:
    html = b.get_attribute('outerHTML')
    idx = html.find('find.gif')
    if idx != -1:
        idx = html.find('checkCollegeName')
        if idx == -1:
            break
html

'<input type="image" name="" src="/cs/page/images/buttons/find.gif" onclick="preferenceButton()" title="FIND" alt="FIND">'

In [47]:
button = b 
button.click()

In [48]:
df = pd.read_html(driver.page_source)

In [49]:
len(df)

5

In [50]:
for d in df:
    print(d.head(3))

                   0       1
0             State:      WA
1  Institution Type:  Public
2                NaN     NaN
   Your College Chances                                       College Name
0                   NaN  Central Washington University OverviewAdmissio...
1                   NaN  Eastern Washington University OverviewAdmissio...
2                   NaN  Evergreen State College OverviewAdmissionMoney...
           City   State        Size Gender Mix Entr.Diff.        Cost  \
  Your NetPrice Res.COA Non-Res.COA    NeedMet   MeritAid StudentDebt   
0    Ellensburg      WA     11047.0       Coed        Mod         NaN   
1        Cheney      WA     11473.0       Coed        NaN         NaN   
2       Olympia      WA      3610.0       Coed        Mod         NaN   

  Financial Friendliness  Outcomes        Type Student Background  \
            Fresh.Satis. Grad.Rate Amer.Indian       AfricanAmer.   
0                $22,547   $36,654         81%                22%   
1          

In [51]:
df[1].loc[0,"College Name"]

'Central Washington University OverviewAdmissionMoney MattersAcademicsCampus LifeStudents'

In [52]:
# pandas confused the button text and the drop down text, need to 
# strip from <space>Overview to end of string
name = df[1].loc[0,"College Name"]
idx = name.find(' Overview')
idx

29

In [53]:
name = name[:idx]
name

'Central Washington University'

In [54]:
thing = driver.find_element_by_link_text(name)
thing

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="0423d527-cdf7-3e44-ac6f-62b84884ceea", element="ddf3b091-2df7-9b40-ba03-23d035be2d81")>

In [55]:
thing.get_attribute('outerHTML')

'<a href="https://www.collegedata.com/cs/data/college/college_pg01_tmpl.jhtml?schoolId=929" onclick="setCookie(\'previousPageSection\',\'page_collegeMatch\');" class="collegelink">Central Washington University<img src="/cs/page/images/icon/arrow_down.gif" class="dropDownIcon"></a>'

In [56]:
href = thing.get_attribute('href')
href

'https://www.collegedata.com/cs/data/college/college_pg01_tmpl.jhtml?schoolId=929'

In [57]:
money_page = href.replace('pg01_tmpl', 'pg03_tmpl')
money_page

'https://www.collegedata.com/cs/data/college/college_pg03_tmpl.jhtml?schoolId=929'

In [58]:
driver.get(money_page)

In [59]:
money_data = pd.read_html(driver.page_source)
len(money_data)

13

In [60]:
for mdf in money_data:
    print(mdf.head(2))

                    0                                       1    2    3    4
0  Cost of Attendance  In-state: $22,547Out-of-state: $36,654  NaN  NaN  NaN
1    Tuition and Fees   In-state: $7,849Out-of-state: $21,956  NaN  NaN  NaN
          0                       1    2    3    4
0    E-mail                     NaN  NaN  NaN  NaN
1  Web Site  Financial Aid Web Site  NaN  NaN  NaN
                      0                                      1    2    3    4
0  Application Deadline           Priority deadline February 1  NaN  NaN  NaN
1    Award Notification  On a rolling basis beginning March 15  NaN  NaN  NaN
         Forms Required Cost to File Unnamed: 2 Unnamed: 3 Unnamed: 4
0  FAFSA Code is 003771         Free        NaN        NaN        NaN
1                     ?            ?          ?          ?          ?
                              0                            1    2    3    4
0      Financial Aid Applicants    1,724 (91.1%) of freshmen  NaN  NaN  NaN
1  Found to Have Fin

In [62]:
print(money_data[6])

                                       0             1    2    3    4
0       2016 Graduates WhoTook Out Loans           84%  NaN  NaN  NaN
1  Average Indebtedness of2016 Graduates       $20,411  NaN  NaN  NaN
2           Parents Borrowing PLUS Loans           NaN  NaN  NaN  NaN
3       Parents of StudentsReceiving Aid  Not reported  NaN  NaN  NaN
4   Parents of Students NotReceiving Aid  Not reported  NaN  NaN  NaN
5                                      ?             ?    ?    ?    ?


In [63]:
driver.close()

In [71]:
city = df[2]['City'].dropna()
city

,Your NetPrice
0,Ellensburg
1,Cheney
2,Olympia
3,Seattle
4,Bothell
5,Tacoma
6,Pullman
7,Bellingham


In [72]:
city.applymap(str.upper)

,Your NetPrice
0,ELLENSBURG
1,CHENEY
2,OLYMPIA
3,SEATTLE
4,BOTHELL
5,TACOMA
6,PULLMAN
7,BELLINGHAM


In [73]:
type(city)

pandas.core.frame.DataFrame

In [75]:
df[2]

,City,State,Size,Gender Mix,Entr.Diff.,Cost,Financial Friendliness,Outcomes,Type,Student Background,Unnamed: 10_level_0,Unnamed: 11_level_0,Unnamed: 12_level_0,Unnamed: 13_level_0,Unnamed: 14_level_0,Unnamed: 15_level_0,Unnamed: 16_level_0,Unnamed: 17_level_0,Unnamed: 18_level_0
,Your NetPrice,Res.COA,Non-Res.COA,NeedMet,MeritAid,StudentDebt,Fresh.Satis.,Grad.Rate,Amer.Indian,AfricanAmer.,Asian/Pac. Is.,Hispan.,Int'l.,Unnamed: 13_level_1,Unnamed: 14_level_1,Unnamed: 15_level_1,Unnamed: 16_level_1,Unnamed: 17_level_1,Unnamed: 18_level_1
0,Ellensburg,WA,11047.0,Coed,Mod,NaN,"$22,547","$36,654",81%,22%,"$20,411",76.0%,27.0%,Pub,0.7%,3.8%,4.6%,15.0%,2.9%
1,Cheney,WA,11473.0,Coed,NaN,NaN,"$22,142","$38,895",86%,32%,"$27,961",75.8%,25.1%,Pub,1.2%,3.6%,3.4%,16.6%,4.5%
2,Olympia,WA,3610.0,Coed,Mod,NaN,"$21,207","$38,667",57%,4%,"$21,147",61.0%,42.2%,Pub,2.5%,4.9%,3.2%,10.9%,0.6%
3,Seattle,WA,31331.0,Coed,Very,NaN,"$25,948","$49,986",79%,20%,"$19,880",94.0%,65.0%,Pub,0.5%,3.5%,28.8%,9.2%,14.6%
4,Bothell,WA,5365.0,Coed,Mod,NaN,"$26,137","$50,175",78%,5%,"$18,850",88.0%,42.9%,Pub,0.3%,7.1%,30.3%,10.6%,9.3%
5,Tacoma,WA,4436.0,Coed,Mod,NaN,"$25,675","$49,713",79%,17%,"$17,480",75.7%,39.7%,Pub,0.8%,8.8%,21.3%,15.5%,4.2%
6,Pullman,WA,17868.0,Coed,Mod,NaN,"$27,249","$41,675",65%,74%,"$26,807",81.7%,38.6%,Pub,0.7%,3.9%,7.2%,14.6%,5.5%
7,Bellingham,WA,14968.0,Coed,Mod,NaN,"$23,984","$38,496",94%,6%,"$15,663",81.0%,37.6%,Pub,0.4%,1.7%,6.7%,9.1%,1.1%
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
df[2].loc[1,'City']

Your NetPrice    Cheney
Name: 1, dtype: object

In [78]:
df[2].columns

MultiIndex(levels=[['City', 'Cost', 'Entr.Diff.', 'Financial Friendliness', 'Gender Mix', 'Outcomes', 'Size', 'State', 'Student Background', 'Type', 'Unnamed: 10_level_0', 'Unnamed: 11_level_0', 'Unnamed: 12_level_0', 'Unnamed: 13_level_0', 'Unnamed: 14_level_0', 'Unnamed: 15_level_0', 'Unnamed: 16_level_0', 'Unnamed: 17_level_0', 'Unnamed: 18_level_0'], ['AfricanAmer.', 'Amer.Indian', 'Asian/Pac. Is.', 'Fresh.Satis.', 'Grad.Rate', 'Hispan.', 'Int'l.', 'MeritAid', 'NeedMet', 'Non-Res.COA', 'Res.COA', 'StudentDebt', 'Unnamed: 13_level_1', 'Unnamed: 14_level_1', 'Unnamed: 15_level_1', 'Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1', 'Your NetPrice']],
           labels=[[0, 7, 6, 4, 2, 1, 3, 5, 9, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18], [18, 10, 9, 8, 7, 11, 3, 4, 1, 0, 2, 5, 6, 12, 13, 14, 15, 16, 17]])

In [80]:
df[2].columns.droplevel(1)

Index(['City', 'State', 'Size', 'Gender Mix', 'Entr.Diff.', 'Cost',
       'Financial Friendliness', 'Outcomes', 'Type', 'Student Background',
       'Unnamed: 10_level_0', 'Unnamed: 11_level_0', 'Unnamed: 12_level_0',
       'Unnamed: 13_level_0', 'Unnamed: 14_level_0', 'Unnamed: 15_level_0',
       'Unnamed: 16_level_0', 'Unnamed: 17_level_0', 'Unnamed: 18_level_0'],
      dtype='object')

In [81]:
df[2].columns = df[2].columns.droplevel(1)
df[2].head()

,City,State,Size,Gender Mix,Entr.Diff.,Cost,Financial Friendliness,Outcomes,Type,Student Background,Unnamed: 10_level_0,Unnamed: 11_level_0,Unnamed: 12_level_0,Unnamed: 13_level_0,Unnamed: 14_level_0,Unnamed: 15_level_0,Unnamed: 16_level_0,Unnamed: 17_level_0,Unnamed: 18_level_0
0,Ellensburg,WA,11047.0,Coed,Mod,NaN,"$22,547","$36,654",81%,22%,"$20,411",76.0%,27.0%,Pub,0.7%,3.8%,4.6%,15.0%,2.9%
1,Cheney,WA,11473.0,Coed,NaN,NaN,"$22,142","$38,895",86%,32%,"$27,961",75.8%,25.1%,Pub,1.2%,3.6%,3.4%,16.6%,4.5%
2,Olympia,WA,3610.0,Coed,Mod,NaN,"$21,207","$38,667",57%,4%,"$21,147",61.0%,42.2%,Pub,2.5%,4.9%,3.2%,10.9%,0.6%
3,Seattle,WA,31331.0,Coed,Very,NaN,"$25,948","$49,986",79%,20%,"$19,880",94.0%,65.0%,Pub,0.5%,3.5%,28.8%,9.2%,14.6%
4,Bothell,WA,5365.0,Coed,Mod,NaN,"$26,137","$50,175",78%,5%,"$18,850",88.0%,42.9%,Pub,0.3%,7.1%,30.3%,10.6%,9.3%
